In [8]:
import torch
import torch.nn as nn

In [9]:
a = torch.randn(3,3)
drop = nn.Dropout(p=0.5)
a

tensor([[-0.9686,  0.2545,  0.3954],
        [ 0.5001, -0.2782, -0.8796],
        [ 1.5114, -0.6909, -1.4617]])

In [10]:
drop(a)

tensor([[-1.9373,  0.0000,  0.7909],
        [ 1.0001, -0.0000, -1.7591],
        [ 3.0229, -0.0000, -0.0000]])

In [11]:
bsz = 3
seqlen= 4
num_experts = 3
dim = 1024
rank=16


In [12]:
x = torch.randn(bsz,seqlen,dim)

In [13]:
linearA = nn.Linear(dim,rank,bias=False)
linearB = nn.Linear(rank,dim,bias=False)
core1 = nn.Parameter(torch.randn(rank,rank))
core2 = nn.Parameter(torch.randn(rank,rank))
core3 = nn.Parameter(torch.randn(rank,rank))
router = nn.Linear(dim,num_experts)

In [14]:
core1

Parameter containing:
tensor([[-1.4418e+00, -3.2661e-01,  4.5046e-01,  5.1338e-01, -1.0101e+00,
          4.7987e-01, -1.5209e+00,  9.0739e-02,  7.9590e-01,  2.6513e-01,
         -1.0685e+00, -1.6210e+00, -9.9646e-01, -2.3865e-01,  1.5378e+00,
         -3.7784e-01],
        [ 1.1735e+00, -3.4900e-01, -3.7348e-01, -1.8383e+00, -7.9980e-01,
          6.5487e-01, -3.7663e-01, -3.2131e-01,  3.3812e-02,  2.3504e+00,
          6.6073e-01,  1.0282e-01, -1.4481e+00,  2.7544e-01, -9.6476e-01,
         -8.7495e-01],
        [ 5.5766e-01, -6.4752e-02,  4.0390e-01,  6.4736e-01, -7.4097e-01,
          1.2309e+00, -2.0013e-01, -6.5332e-01,  9.5750e-01,  1.6662e+00,
          3.7990e-02,  3.1834e-01, -6.9204e-01, -6.2695e-01,  5.3933e-01,
          2.2109e-01],
        [-1.0784e+00, -1.0771e+00, -1.3390e-01,  6.7201e-01, -6.2751e-01,
         -1.4927e-01,  1.6580e+00,  1.3145e+00, -9.7420e-01, -2.7986e-01,
          1.2557e+00,  2.2012e+00,  1.1231e+00,  1.0971e-01, -1.6548e+00,
         -6.8548e-02]

In [15]:
router_weight = router(x)

In [16]:
router_weight.shape

torch.Size([3, 4, 3])

In [17]:
t = linearA(x)
h = linearB(t @ core1) * router_weight[:,:,0].unsqueeze(-1) + linearB(t @ core2) * router_weight[:,:,1].unsqueeze(-1) + linearB(t @ core3) * router_weight[:,:,2].unsqueeze(-1)

In [18]:
h

tensor([[[ 0.2837,  0.2740,  1.6536,  ..., -0.7547, -0.7401,  0.3571],
         [-1.4385,  1.3821,  0.3053,  ...,  2.8384,  2.8448, -4.2237],
         [ 0.8881, -0.6445, -0.8536,  ..., -2.3374,  0.7740,  0.7516],
         [ 0.4627, -0.5836, -0.5366,  ...,  0.0942, -0.6412,  0.4636]],

        [[ 0.4807,  0.8200,  0.4833,  ...,  0.3043,  0.2149, -0.8079],
         [-0.3117, -0.6264, -0.3601,  ...,  1.4480, -0.6694, -0.4156],
         [ 2.3721, -0.5174, -1.2260,  ...,  0.3132, -0.7961,  1.5619],
         [-0.3477, -1.9478,  0.8626,  ...,  2.5336, -1.4773,  0.1803]],

        [[-0.4747,  0.1713,  0.4737,  ...,  0.0144,  0.2497, -1.9615],
         [-1.2913, -1.0215, -0.7058,  ..., -1.3398,  0.2412,  2.1453],
         [-0.3390, -0.2092,  0.2340,  ..., -0.2674,  0.1017,  0.0084],
         [-1.1695,  1.6520,  2.2143,  ...,  2.1914, -2.4855,  0.6907]]],
       grad_fn=<AddBackward0>)

In [23]:
test = core1 * router_weight[:,:,0].unsqueeze(-1).unsqueeze(-1)

In [24]:
test.shape

torch.Size([3, 4, 16, 16])

In [19]:
core = (core1 * router_weight[:,:,0].unsqueeze(-1).unsqueeze(-1) + core2 * router_weight[:,:,1].unsqueeze(-1).unsqueeze(-1) + core3 * router_weight[:,:,2].unsqueeze(-1).unsqueeze(-1))

In [25]:
core.shape

torch.Size([3, 4, 16, 16])

In [39]:
h2_=torch.einsum('bsrr,bsr->bsr',core,linearA(x) )

In [40]:
h3_ = linearB(h2_)

In [41]:
h3_

tensor([[[-0.1329,  0.2742, -0.3628,  ..., -0.2147, -0.1812,  0.4554],
         [ 0.2041,  0.0353, -0.5267,  ..., -0.1674,  0.2409,  0.8286],
         [-0.0753,  0.0321,  0.1317,  ..., -0.1282,  0.0046,  0.0396],
         [ 0.9561, -0.0449,  0.0016,  ..., -0.0255,  0.3517,  0.0215]],

        [[ 0.5108, -0.6670, -0.1952,  ..., -0.0209, -0.1151,  0.2315],
         [-0.3230,  0.2307, -0.6154,  ...,  0.0365, -0.1763,  0.0686],
         [-0.2336, -0.1466,  0.0708,  ..., -0.0599, -0.1147,  0.0115],
         [ 0.0760, -0.0658, -0.0462,  ..., -0.2356,  0.0167,  0.3105]],

        [[ 0.4001,  0.0139, -0.1037,  ..., -0.0468,  0.6063, -0.2202],
         [-0.0031,  0.0571,  0.0074,  ..., -0.0494,  0.1037, -0.1060],
         [ 0.1458, -0.0303, -0.1922,  ...,  0.2921, -0.1696, -0.0940],
         [-0.0958,  0.3255,  0.3887,  ..., -0.0987, -0.1626, -0.3898]]],
       grad_fn=<UnsafeViewBackward0>)

In [29]:
core.shape

torch.Size([3, 4, 16, 16])

In [28]:
h2 = linearA(x).unsqueeze(-2) @ core

RuntimeError: The size of tensor a (3) must match the size of tensor b (4) at non-singleton dimension 1

In [29]:
h3 = linearB(h2.squeeze(-2))

In [30]:
h3

tensor([[[ 0.2837,  0.2740,  1.6536,  ..., -0.7547, -0.7401,  0.3571],
         [-1.4385,  1.3821,  0.3053,  ...,  2.8384,  2.8448, -4.2237],
         [ 0.8881, -0.6445, -0.8536,  ..., -2.3374,  0.7740,  0.7516],
         [ 0.4627, -0.5836, -0.5366,  ...,  0.0942, -0.6412,  0.4636]],

        [[ 0.4807,  0.8200,  0.4833,  ...,  0.3043,  0.2149, -0.8079],
         [-0.3117, -0.6264, -0.3601,  ...,  1.4480, -0.6694, -0.4156],
         [ 2.3721, -0.5174, -1.2260,  ...,  0.3132, -0.7961,  1.5619],
         [-0.3477, -1.9478,  0.8626,  ...,  2.5336, -1.4773,  0.1803]],

        [[-0.4747,  0.1713,  0.4737,  ...,  0.0144,  0.2497, -1.9615],
         [-1.2913, -1.0215, -0.7058,  ..., -1.3398,  0.2412,  2.1453],
         [-0.3390, -0.2092,  0.2340,  ..., -0.2674,  0.1017,  0.0084],
         [-1.1695,  1.6520,  2.2143,  ...,  2.1914, -2.4855,  0.6907]]],
       grad_fn=<UnsafeViewBackward0>)

In [ ]:
linearA(x).shape

torch.Size([3, 4, 16])

: 

In [ ]:
h2.shape

torch.Size([3, 4, 1, 16])

: 

In [ ]:
linearA(x).unsqueeze(-2).shape

torch.Size([3, 4, 1, 16])

: 

In [2]:
a = torch.load('/data/workspace/projects/moe/AdaMoLE/outputs/qwen3-8b-mocorelorash-exp8-corerouter-math-14k/adapter_model.bin')

In [3]:
a.keys()

dict_keys(['base_model.model.model.layers.0.self_attn.q_proj.lora_Cores.0', 'base_model.model.model.layers.0.self_attn.q_proj.lora_Cores.1', 'base_model.model.model.layers.0.self_attn.q_proj.lora_Cores.2', 'base_model.model.model.layers.0.self_attn.q_proj.lora_Cores.3', 'base_model.model.model.layers.0.self_attn.q_proj.lora_Cores.4', 'base_model.model.model.layers.0.self_attn.q_proj.lora_Cores.5', 'base_model.model.model.layers.0.self_attn.q_proj.lora_Cores.6', 'base_model.model.model.layers.0.self_attn.q_proj.lora_Cores.7', 'base_model.model.model.layers.0.self_attn.q_proj.lora_router.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_Cores.0', 'base_model.model.model.layers.0.self_attn.k_proj.lora_Cores.1', 'base_model.model.model.layers.0.self_attn.k_proj.lora_Cores.2', 'base_model.model.model.layers.0.self_attn.k_proj.lora_Cores.3', 'base_model.model.model.layers.0.self_attn.k_proj.lora_Cores.4', 'base_model.model.model.layers.0.self_attn.k_proj.lora_Cores.5', 'base_mo

In [4]:
b = [x for x in a.keys() if 'share' in x]

In [5]:
b.__len__()

12

In [15]:
c = [x for x in a.keys() if 'drop' in x]

In [7]:
36*12+399


831

In [4]:
a['base_model.model.model.layers.0.self_attn.q_proj.moe_layer.experts.0.lora_A.weight']

tensor([[-4.4629e-03, -2.4648e-03, -7.0186e-03,  ...,  1.6958e-02,
         -1.3726e-03,  4.9094e-03],
        [ 4.5847e-03, -1.2500e-02, -1.3889e-02,  ...,  6.0564e-03,
          1.4926e-02, -4.6238e-03],
        [ 9.3714e-03, -1.5020e-02, -1.2793e-02,  ...,  8.6024e-03,
         -3.0532e-03, -8.7763e-03],
        ...,
        [ 9.4404e-03, -3.7128e-04,  6.3849e-03,  ...,  4.1793e-03,
         -1.0146e-02,  6.7490e-04],
        [-3.4635e-03, -6.5786e-03,  1.6937e-02,  ...,  1.2113e-02,
         -6.3088e-03,  1.0748e-02],
        [ 2.2891e-03,  4.8487e-03, -8.5005e-06,  ..., -1.1082e-03,
          8.0367e-03,  1.5279e-03]], device='cuda:0')

In [5]:
a['base_model.model.model.layers.0.self_attn.q_proj.lora_A.0.weight']

tensor([[-4.4629e-03, -2.4648e-03, -7.0186e-03,  ...,  1.6958e-02,
         -1.3726e-03,  4.9094e-03],
        [ 4.5847e-03, -1.2500e-02, -1.3889e-02,  ...,  6.0564e-03,
          1.4926e-02, -4.6238e-03],
        [ 9.3714e-03, -1.5020e-02, -1.2793e-02,  ...,  8.6024e-03,
         -3.0532e-03, -8.7763e-03],
        ...,
        [ 9.4404e-03, -3.7128e-04,  6.3849e-03,  ...,  4.1793e-03,
         -1.0146e-02,  6.7490e-04],
        [-3.4635e-03, -6.5786e-03,  1.6937e-02,  ...,  1.2113e-02,
         -6.3088e-03,  1.0748e-02],
        [ 2.2891e-03,  4.8487e-03, -8.5005e-06,  ..., -1.1082e-03,
          8.0367e-03,  1.5279e-03]], device='cuda:0')

In [10]:
a.keys().__len__()

5940